In [1]:
%matplotlib inline

from pathlib import Path
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append(os.path.join(os.getcwd(), os.pardir, 'src'))

%load_ext autoreload

%autoreload 2
from visualization.visualize import *

# Load WHO Data

In [3]:
who_df = pd.read_csv(Path("../data/interim/calc_cols_added.csv"), index_col=0)
who_df.head(2)

,Iso Code,Country Name,WHO Region,Year,Vaccine Type,Admin1,Admin2,DenomType,Denominator,Numerator,Coverage,recalc_numerator,available_admin,normalized_country,indicator,group,vaccine,timing,reported
0,AFG,Afghanistan,EMRO,2016,BCG,NaN,Aab Band,1.0,1266.0,NaN,51.801245,655.803767,Aab Band,afghanistan,1,BCG,BCG,1st_birth,True
1,AFG,Afghanistan,EMRO,2016,BCG,NaN,Aab Kamari,1.0,4599.0,NaN,94.674670,4354.088093,Aab Kamari,afghanistan,1,BCG,BCG,1st_birth,True


In [4]:
# These countries appear in both the MICS and the WHO datasets
overlaps = [
    'Benin',
    'Cameroon',
    'Cuba',
    'Dominican Republic',
    'El Salvador',
    'Guyana',
    'Kazakhstan',
    'Kyrgyzstan',
    'Malawi',
    'Mali',
    'Mexico',
    'Nepal',
    'Paraguay',
    'Sudan',
    'Swaziland',
    'Turkmenistan',
    'Zimbabwe',
]

# some MICS regions are just subsets of a country, so we
# can only analyze the ones that appear in both datasets
partial_matches = {
    'Kenya (Bungoma County)': 'Kenya',
    'Kenya (Kakamega County)': 'Kenya',
    'Kenya (Turkana County)': 'Kenya',
    'Pakistan (Punjab)': 'Pakistan',
    'Pakistan (Sindh)': 'Pakistan',
}

# Some MICS countries have their names written differently
# Thus maps the MICS version as the key to the WHO version
# as the value
overlap_map = {
    'Guinea Bissau': 'Guinea-Bissau',
    'Sao Tome and Principe': 'Sao Tome & Principe',
}

all_mics_overlaps = list(overlap_map.keys()) + list(partial_matches.keys()) + overlaps

# Load MICS Survey data

In [5]:
childrens_surveys = dict()

for f in Path("../data/interim/MICS").iterdir():
    if f.name.startswith('.'):
        continue
    elif not f.is_dir():
        continue
    childrens_surveys[f.stem] = pd.read_csv(f/'ch.csv', index_col=0)

/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14,21,30,32,35,37,45,57,77,79,82,98,101,103,104,106,145,149,153,174,175,176,177,178,180,181,182,183,184,185,186,187,188,189,199,200,201,203,204,205,206,208,219,220,221,222,223,224,225,226,227,228,230,231,232,238,239,240,241,246,249,266,267,268,270,281,291,300) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (245,253,254,255,265,289,345) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15,16,17,18,78,81,

/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15,16,17,18,78,81,145,152,174,177,179,181,185,187,203,205,206,207,219,226,230,232,286,314,316) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,25,103,104,105,106,107,108,176,178,183,197,198,199,202,213,214,217,229,231,233,235,240,251,253,279) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15,16,17,18,19,101,102,103,170,181,193,200,264,290) have mixed types. Specify dtype option on import or set lo

### Upsample surveys based on child weights to get representative child counts

In [6]:
childrens_surveys_upsampled = dict()
for country, df in childrens_surveys.items():
    rng = np.random.RandomState(12345)
    
    w_col = 'chweight' if 'chweight' in df.columns else 'CHWEIGHT'
    
    smpl = rng.choice(df.index, 100000, p=df[w_col] / df[w_col].sum())
    childrens_surveys_upsampled[country] = df.loc[smpl, :]

In [7]:
relevant_vars = [
    # When conducted
    'UF8Y',
    
    # Where conducted (urban/rural)
    'HH6',
    'HH7',
    
    # Child Age
    'AG2',
    'CAGE',
    'AG1Y',
]

In [8]:
subsetted_dfs = []
for c in all_mics_overlaps:
    df = childrens_surveys_upsampled[c]
    im_vars = [c for c in df.columns if c.startswith('IM')]
    
    df['country'] = c
    subsetted_dfs.append(df[['country'] + im_vars + relevant_vars].copy())

/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
all_mics_children_data = pd.concat(subsetted_dfs)

In [10]:
subsetted_dfs = []
for c in all_mics_overlaps:
    df = childrens_surveys[c]
    im_vars = [c for c in df.columns if c.startswith('IM')]
    
    df['country'] = c
    subsetted_dfs.append(df[['country'] + im_vars + relevant_vars].copy())

raw_mics_children_data = pd.concat(subsetted_dfs)

In [ ]:
# nepal is in nepalese years...
all_mics_children_data.replace({'UF8Y': {2070.: 2013., 2071.: 2014.}}, inplace=True)
raw_mics_children_data.replace({'UF8Y': {2070.: 2013., 2071.: 2014.}}, inplace=True)

In [ ]:
# just include surveys done in at least 2014
min_year = all_mics_children_data.groupby('country').UF8Y.min()
to_include = min_year[min_year >= 2014].index.tolist()

mics_children_data_who_countries = all_mics_children_data[all_mics_children_data.country.isin(to_include)]
raw_mics_children_data_who_countries = raw_mics_children_data[raw_mics_children_data.country.isin(to_include)]

print(mics_children_data_who_countries.shape)
mics_children_data_who_countries.dropna(how='all', axis=1, inplace=True)
raw_mics_children_data_who_countries = raw_mics_children_data_who_countries[mics_children_data_who_countries.columns]
print(mics_children_data_who_countries.shape)

In [ ]:
country_year_map = mics_children_data_who_countries.groupby('country').UF8Y.max().to_dict()

mics_children_data_who_countries['survey_year'] = mics_children_data_who_countries.country.replace(country_year_map)
raw_mics_children_data_who_countries['survey_year'] = \
    raw_mics_children_data_who_countries.country.replace(country_year_map)

In [ ]:
from collections import namedtuple

Vax = namedtuple('Vax',
                 ['name',           # Group name
                  'doses_dict',     # map standard WHO dose names to year columns
                  'ever_col',       # yes/no for vax ever given 
                  'ever_col_year']) # year for vax ever given

vaxes = [
    Vax(name='BCG',
        doses_dict={'BCG': ['IM3BY']},
        ever_col=['IM7'],
        ever_col_year=[]),
    
    # DTP or Penta
    Vax(name='DTP',
        doses_dict={'DTP1': ['IM3D1Y', 'IM3PV1Y'],
                    'DTP2': ['IM3D2Y', 'IM3PV2Y'],
                    'DTP3': ['IM3D3Y', 'IM3PV3Y'],
                    'DTP4': ['IM3D4Y', 'IM3PV4Y']},
        ever_col=['IM17', 'IM17A', 'IM11', 'IM10A', 'IM11A'],
        ever_col_year=[]),
    
    Vax(name='Measles',
        doses_dict={'MCV1': ['IM3M1Y'],
                    'MCV2': ['IM3M2Y']},
        ever_col=['IM16'],
        ever_col_year=['IM3MY']),
    
# polio
# flu/Hib
# hepb
# pneumo / PCV
]

In [ ]:
# Harmonize "ever" questions:
ever_harmonize_map = {
    # yes
    'First two weeks': 1.0,
    'First eight weeks': 1.0,
    'Dans les 2 premières semaines': 1.0,
    'Later': 1.0,
    'Plus tard': 1.0,
    'Sim': 1.0,
    'Si': 1.0,
    'Sí': 1.0,
    'Oui': 1.0,
    'Yes': 1.0,
    '1.0': 1.0,
    '2.0': 1.0,
    '3.0': 1.0,
    '4.0': 1.0,
    '5.0': 1.0,
    '6.0': 1.0,
     2.0: 1.0,
     3.0: 1.0,
     4.0: 1.0,
     '2009.0': 1.0,
     '2010.0': 1.0,
     '2011.0': 1.0,
     '2012.0': 1.0,
     '2013.0': 1.0,
     '2014.0': 1.0,
     '2015.0': 1.0,
     '2016.0': 1.0,
     2013.0: 1.0,
     2014.0: 1.0,
     2015.0: 1.0,
     2016.0: 1.0,  
    'Dans carte de vaccination': 1.0,
    'Déclaration de la mère': 1.0,
    'Madre reportada': 1.0,
    'Reporte de la madre': 1.0,
    'Réponse de la mère': 1.0,
    'Marcada en la tarjeta': 1.0,
    'Marcada sobre la tarjeta': 1.0,
    'Marked on card': 1.0,
    'Marqué sur le carnet': 1.0,
    'Mother reported': 1.0,
    
    # no
    'Non': 0.0,
    'NÃ£o': 0.0,
    'No': 0.0,
    'No aplicada': 0.0,
    'No vacunado': 0.0,
    'Pas reçue/pas vue dans le carnet': 0.0,
    
    # don't know
    'DK': np.nan,
    'Manquant': np.nan,
    'Non déclaré/Pas de réponse': np.nan,
    'Ne sait pas': np.nan,
    'Missing': np.nan,
    'Em falta': np.nan,
    'Incohérent': np.nan,
    'Inconsistent': np.nan,
    'Inconsistente': np.nan,
    'NS': np.nan,
    'NSP': np.nan,
    'No dada': np.nan,
    'No sabe': np.nan,
    'Not given': np.nan,
    'NÃ£o Sabe': np.nan,
    'Omitido': np.nan,
    'Pas donné': np.nan,
}

In [ ]:
vaxed_data = mics_children_data_who_countries.copy()
raw_vaxed_data = raw_mics_children_data_who_countries.copy()

## All one year olds

In [ ]:
MIN_AGE = 0
MAX_AGE = 24

# calc denominator
vaxed_data[f"denom"] = (vaxed_data.CAGE >= MIN_AGE) & (vaxed_data.CAGE <= MAX_AGE)
raw_vaxed_data[f"denom"] = (raw_vaxed_data.CAGE >= MIN_AGE) & (raw_vaxed_data.CAGE <= MAX_AGE)

vaxed_data = vaxed_data[vaxed_data.denom]
raw_vaxed_data = raw_vaxed_data[raw_vaxed_data.denom]

In [ ]:
vaxed_data

In [ ]:
all_cols_added = ["denom"]

def get_ever_based_on_any_dose(dd, ever, ever_yr, ca):
    return (((dd[ca] == 1.0).any(axis=1) |    # any dose that we added
             (dd[ever_yr].replace(ever_harmonize_map) == 1).any(axis=1) | # any dose marked with year
             (dd[ever].replace(ever_harmonize_map) == 1).any(axis=1)) & # any dose recall
             dd[f"denom"])
    
#     return (((vaxed_data[cols_added] == 1.0).any(axis=1) |    # any
#               pd.notnull(vaxed_data[v.ever_col_year]).any(axis=1) | 
#               (vaxed_data[v.ever_col].replace(ever_harmonize_map) == 1.0).any(axis=1)) &
#              vaxed_data[f"denom"])

for v in vaxes:

    cols_added = []

    print(v.name)

    # track individual doses
    for dose_name, dose_cols in v.doses_dict.items():

        print(dose_name)
        new_col = f"mics_{dose_name}"
        vaxed_data[new_col] = pd.notnull(vaxed_data[dose_cols]).any(axis=1) & vaxed_data[f"denom"]
        raw_vaxed_data[new_col] = pd.notnull(raw_vaxed_data[dose_cols]).any(axis=1) & raw_vaxed_data[f"denom"]

        cols_added.append(new_col)

    # track ever based on ever_col or individual doses
    new_col = f"mics_{v.name}_any_dose"    
    vaxed_data[new_col] = get_ever_based_on_any_dose(vaxed_data, v.ever_col, v.ever_col_year, cols_added)    
    raw_vaxed_data[new_col] = get_ever_based_on_any_dose(raw_vaxed_data, v.ever_col, v.ever_col_year, cols_added)

    all_cols_added += cols_added + [new_col]

#### Deprecated per-year calculations (too few children)

In [ ]:
# all_cols_added = dict()

# for ix, yr in enumerate(list(range(2016, 2013, -1))):
    
#     all_cols_added[yr] = [f"denom_{yr}"]
    
#     # calc denominator
#     vaxed_data[f"denom_{yr}"] = vaxed_data.AG1Y == yr
#     raw_vaxed_data[f"denom_{yr}"] = raw_vaxed_data.AG1Y == yr

#     for v in vaxes:
        
#         cols_added = []
        
#         print(v.name)
        
#         # track individual doses
#         for dose_name, dose_cols in v.doses_dict.items():
#             print(dose_name)
            
#             new_col = f"{dose_name}_{yr}"
#             vaxed_data[new_col] = (vaxed_data[dose_cols] == yr).any(axis=1) & vaxed_data[f"denom_{yr}"]
#             raw_vaxed_data[new_col] = (raw_vaxed_data[dose_cols] == yr).any(axis=1) & raw_vaxed_data[f"denom_{yr}"]
            
#             cols_added.append(new_col)
        
#         # track ever based on ever_col or individual doses
#         new_col = f"{v.name}_any_dose_{yr}"
        
#         vaxed_data[new_col] = (((vaxed_data[cols_added] == 1.0).any(axis=1) | 
#                                 (vaxed_data[v.ever_col_year] == yr).any(axis=1) | 
#                                 (vaxed_data[v.ever_col].replace(ever_harmonize_map) == 1.0).any(axis=1)) &
#                                vaxed_data[f"denom_{yr}"])
        
#         raw_vaxed_data[new_col] = (((raw_vaxed_data[cols_added] == 1.0).any(axis=1) | 
#                         (raw_vaxed_data[v.ever_col_year] == yr).any(axis=1) | 
#                         (raw_vaxed_data[v.ever_col].replace(ever_harmonize_map) == 1.0).any(axis=1)) &
#                        raw_vaxed_data[f"denom_{yr}"])

#         all_cols_added[yr] += cols_added + [new_col]

In [ ]:
sampled_data = vaxed_data.groupby(['country', 'HH7'])[all_cols_added].sum()
raw_data = raw_vaxed_data.groupby(['country', 'HH7'])[all_cols_added].sum()

# calculate coverage
original_denom = sampled_data[f"denom"].copy()
cov_percentages = sampled_data.divide(original_denom, axis=0)
cov_percentages.denom = original_denom

# only show data where we talked to enough kids
mics_to_compare = cov_percentages.loc[raw_data[f"denom"] >= 20, :].reset_index()

print(mics_to_compare.shape)
mics_to_compare.head(2)

In [ ]:
### Create immunization value counts

In [ ]:
who.head()

In [ ]:
who = pd.read_csv(Path("../data/interim/who_subnational_with_geocodes.csv"), index_col=0)


# for v in ['DTP']:
#     who[f"who_any_{v}"] = who[[c for c in who.columns if c.startswith(v)]].max(axis=1)

coverage_by_Admin1 = (who.groupby(['Country Name', 'google_admin_1', 'Vaccine Type'])
                                 [['recalc_numerator', 'Denominator']]
                                    .sum()
                                    .pipe(lambda x: x.recalc_numerator / x.Denominator)
                                    .unstack())

coverage_by_Admin1.reset_index(inplace=True)
coverage_by_Admin1.describe()

## Geocode from `geocodes.json`

In [ ]:
import json
with open(Path("../data/interim/MICS/geocodes.json"), 'r') as f:
    lookups = json.load(f)

In [ ]:
def extract_canonical_location_info(item):
    place_id, google_admin_1, google_admin_2 = None, None, None
    
    if item['results']:
        place_id = item['results'][0]['place_id']
        
        for ac in item['results'][0]['address_components']:
            if 'administrative_area_level_1' in ac['types']:
                google_admin_1 = unidecode(ac['long_name'])
            elif 'administrative_area_level_2' in ac['types']:
                google_admin_2 = unidecode(ac['long_name'])
    
    return {'google_admin_1': google_admin_1,
            'google_admin_2': google_admin_2,
            'place_id': place_id}


In [ ]:
from unidecode import unidecode

In [ ]:
reses = []
for ix, item in mics_to_compare.iterrows():
    
    reses.append(extract_canonical_location_info(lookups[item.country][item.HH7]))


mics_to_compare['google_admin_1'] = pd.DataFrame(reses).google_admin_1
mics_to_compare.head()

In [ ]:
who_and_mics_by_admin1 = mics_to_compare.merge(coverage_by_Admin1,
                                               left_on=['country', 'google_admin_1'],
                                               right_on=['Country Name', 'google_admin_1'],
                                               how='inner')

print(who_and_mics_by_admin1.shape)

In [ ]:
who_and_mics_by_admin1.head()

In [ ]:
pd.set_option("display.max_columns", 100)
who_and_mics_by_admin1.describe()

In [ ]:
# for v in ['DTP', 'MCV']:
#     cols = [c for c in who_and_mics_by_admin1.columns if c.startswith(v)]

# who_and_mics_by_admin1[cols].max(axis=1)

In [ ]:
plot_together = [
    ('DTP', 'mics_DTP_any_dose', "DTP Immunization Coverage"),
    ('BCG', 'mics_BCG_any_dose', "BCG Immunization Coverage"),
    ('MCV', 'mics_Measles_any_dose', "Measles Immunization Coverage"),
]


for who_prefix, mics_prefix, title in plot_together:
    to_plot = pd.DataFrame({
        'WHO Data': who_and_mics_by_admin1[[c for c in who_and_mics_by_admin1.columns if c.startswith(who_prefix)]].fillna(0).max(axis=1).values,
        'MICS Data': who_and_mics_by_admin1[mics_prefix].values,
        },
        index = (who_and_mics_by_admin1.country + ", " + who_and_mics_by_admin1.google_admin_1).values
    )
    
    with styled_fig_ax(size='custom', font_size=8, x_formatter=pct_func_formatter(), zero_lines=False,
                       other_rc_params={'figure.figsize': (3, 6.5)}) as ax:
        to_plot.plot.barh(ax=ax)
        av_err = to_plot.diff(axis=1).abs().mean().values[1]
        ax.set_title(title + " (Mean Error: {:.0f}%)".format(av_err*100))


In [ ]:
who_df[((who_df['Country Name'] == 'Paraguay') & 
        (who_df['Vaccine Type'] == 'BCG'))]

In [ ]:
# upsampled = upsample_dhs(rwa_dhs)

# upsampled.sdistrict = clean_uga_districts(upsampled.sdistrict)
# who_rwa.Admin2 = clean_who_rwa_names(who_rwa.Admin2)



# vax_rates = calculate_rate(upsampled, vaxes)
# unsampled_vax_rates = calculate_rate(rwa_dhs, vaxes)

# district_sums = calculate_district_sums(upsampled, rwa_dhs, vax_rates, unsampled_vax_rates)

# coverage_by_Admin2 = get_admin2_coverage(who_rwa)
# bar_plot_who_dhs(coverage_by_Admin2, district_sums)
violin_plot_who_dhs(coverage_by_Admin2,
                    district_sums,
                    "Rwanda Distribution of Vaccine Coverage Aross Regions",
                    ylim=(0, 2.0))


In [ ]:
# from unidecode import unidecode

# def clean_geo_names(x):
#     x = unidecode(x.strip().upper())
#     x = x.replace("-", " ")
#     return x

In [ ]:
# yr=2014
# df=coverage_dfs[0]

# subset_who = coverage_by_Admin1[coverage_by_Admin1['Country Name'].isin(df.country.unique())]
# subset_who['Admin1'] = subset_who['Admin1'].apply(clean_geo_names)


# subset_who.columns = [f"{c}_{yr}" for c in subset_who.columns]
# subset_who.rename(columns={f'Country Name_{yr}': 'country', f'Admin1_{yr}': 'HH7'}, inplace=True)


# df.HH7 = df.HH7.apply(clean_geo_names)

# shared_regions = np.intersect1d(df.HH7.unique(), subset_who.HH7.unique())

# df = df.sort_values(['country', 'HH7'])[df.HH7.isin(shared_regions)]
# subset_who = subset_who[subset_who.HH7.isin(shared_regions)].sort_values(['country', 'HH7'])

In [ ]:
df

In [ ]:
subset_who